In [3]:
from util import info

import pandas as pd
import numpy as np

In [4]:
df_train = pd.read_csv('train.csv', index_col=[0])
df_test = pd.read_csv('test.csv', index_col=[0])

df = pd.concat([df_train, df_test])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 1 to 1309
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    float64
 1   Pclass    1309 non-null   int64  
 2   Name      1309 non-null   object 
 3   Sex       1309 non-null   object 
 4   Age       1046 non-null   float64
 5   SibSp     1309 non-null   int64  
 6   Parch     1309 non-null   int64  
 7   Ticket    1309 non-null   object 
 8   Fare      1308 non-null   float64
 9   Cabin     295 non-null    object 
 10  Embarked  1307 non-null   object 
dtypes: float64(3), int64(3), object(5)
memory usage: 122.7+ KB


## Data Exploration Notes:
--- 
1. Null values:
    * Age = 263
    * Cabin = 1014 
        * 77% of Cabin data is Null values [Drop / find some utility]  
    * Fare has 1 null value in test training set
2. Text Data:
    * Name
    * Sex
    * Ticket
    * Cabin
    * Embarked
3. Numeric Data:
    * Survived
    * Pclass
    * Age
    * SibSp
    * Parch
    * Fare 
---    
#### Thoughts: ####  
    * Survived mean 0.39, unbalanced Survived ratio?
    * Pclass is skewed more observations with values of 3
    * Age min 0.17 ? 
    * Age mean 30, fairly low considering max value is 80
    * SibSp, Parch and Fare is skewed to the right df.SubSp.hist()
       
#### Value Counts on objects: ####
    * Sex Double the amount of males than females on-board...
    * Name Dubplicates in Test data:  
        * Connolly, Miss. Kate
        * Kelly, Mr. James
    * Tickets lots of values dummy encoding will just make tons of unneeded features...
        * Maybe extract only the numbers and store as int?
        * What about values that don't contain any number?
    * Cabin 186 observations to dummy encode? 
        * Is there a better solution
    * Embarked has 3 Features...
        * Good for dummy encoding however 'S' has triple the amount of occurences than other 2 features combined

In [5]:
df_clean = df.copy()

# Convert sex from object to integer
df_clean.Sex.replace('male', 1, inplace=True)
df_clean.Sex.replace('female', 0, inplace=True)

df_clean['Title'] = [ i.rsplit(', ')[1].rsplit('.')[0] for i in df_clean.Name.values]  # Extract titles

uncommon_titles = 'Rev,Dr,Col,Major,Mlle,Ms,Sir,Capt,Mme,Jonkheer,Lady,the Countess,Don,Dona'.split(',')  
df_clean['Title'] = ['uncommon' if i in uncommon_titles else i for i in df_clean.Title.values]  # Group uncommon titles

# df_clean.drop('Name', axis=1, inplace=True)  # No more 'value' to be extracted from name
df_clean.drop('Cabin', axis=1, inplace=True)  # 77% NaN anyway we can impute data based on fare and title?

null_age = df_clean[df_clean.Age.isnull()].copy()

#  TODO: Impute median age based on Pclass and Sex
# for pclass in df_clean.Pclass.unique():
#         for gender in df_clean.Sex.unique():
#             subset_sex_pclass = df_clean[(df_clean.Sex == gender) & (df_clean.Pclass == pclass)]
#             impute_value = round(subset_sex_pclass.Age.median(), 1)
            
#             for index, row in subset_sex_pclass.iterrows():
#                 df_clean.loc[index, 'Age'] = impute_value
                
